In [ ]:
!pip install pandas transformers guidance numpy evaluate rouge-score nltk py-rouge accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 82.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.

In [ ]:
import pandas as pd
from sklearn.metrics import f1_score, accuracy_score
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch
import json
import nltk
import numpy as np
from guidance import gen
from evaluate import load
import csv
import nltk.translate.bleu_score as bleu
from rouge import Rouge
from transformers import BertTokenizer, BertModel
import torch
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import warnings
from typing import Any, Dict, Tuple
import re
import os

warnings.filterwarnings("ignore", category=UserWarning, message="A new version of the following files was downloaded from")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


# Load Dataset

- id: unique identifier of the example
- source: original model input for paraphrase generation
- hyp1: first alternative paraphrase of the source
- hyp2: second alternative paraphrase of the source
- label: hyp1 or hyp2, based on which of those has been annotated as hallucination
- type: hallucination category assigned.
  - Possible values: addition, named-entity, number, conversion, date, tense, negation, gender, pronoun, antonym, natural

## Trial dataset

This is a small list of examples, provided to help the participants get familiar with the task. Each example contains the following fields:
- id
- source
- hyp1
- hyp2
- label
- type

In [ ]:
from datasets import load_dataset

#load the trial data for both English and Swedish
trial_ds = load_dataset("Eloquent/HalluciGen-PG", name="trial")

#load the trial data only for Swedish
trial_ds_sv = load_dataset("Eloquent/HalluciGen-PG", name="trial", split="trial_swedish")

print(trial_ds.keys())

trial_df_en = trial_ds['trial_english'].to_pandas()
trial_df_se= trial_ds['trial_swedish'].to_pandas()

print('English:')

display(trial_df_en.head())

print('Swedish')
display(trial_df_se.head())

Generating trial_swedish split:   0%|          | 0/20 [00:00<?, ? examples/s]

Generating trial_english split:   0%|          | 0/16 [00:00<?, ? examples/s]

dict_keys(['trial_swedish', 'trial_english'])
English:


,id,source,type,hyp1,hyp2,label
0,0,The population has declined in some 210 of the...,antonym,In the majority of Sweden's 280 municipalities...,In the majority of Sweden's 280 municipalities...,hyp1
1,1,The draft agenda as drawn up by the Conference...,negation,The Conference of Presidents hasn't distribute...,The Conference of Presidents has distributed t...,hyp1
2,2,I am always grateful for comments and suggesti...,antonym,I think I need to listen to everyone's views i...,I think I need to listen to everyone's views i...,hyp1
3,3,The fact is that a key omission from the propo...,named entity,Agenda 2030 does not include a chapter on rene...,Agenda 2000 does not include a chapter on rene...,hyp1
4,4,"We need quite specific legislative proposals, ...",natural,Legislative proposals based on the Treaty of A...,Legislative proposals based solely on the Trea...,hyp2


Swedish


,id,source,type,hyp1,hyp2,label
0,0,"Men intäkterna från mjukvarulicenser, ett mått...",number,"Intäkter från programvarulicenser, en metrik s...","Intäkter från programvarulicenser, en metrik s...",hyp2
1,1,Hongkong-universitet samarbetar med universite...,natural,University of Hong Kong samarbetar med olika k...,University of Hong Kong samarbetar med olika k...,hyp1
2,2,Län med befolkningsminskning kommer att vara V...,named entity,"Vermillion, Posey och Madison är län som komme...","Vermillion, Posey och Marion är län som kommer...",hyp2
3,3,Google presenterar en prototyp för en självkör...,addition,En prototyp för en självkörande bil presentera...,Enligt ett blogginlägg från Google har företag...,hyp2
4,4,Lagförslaget säger att en kvinna som genomgår ...,negation,Förslaget stadgar att en kvinna som genomgår e...,Förslaget stadgar att en kvinna som genomgår e...,hyp1


In [ ]:
#load the test data for the generation step in both English and Swedish
data = load_dataset("Eloquent/HalluciGen-PG", "test_detection")

print(data.keys())

data_df_en = data['test_detection_english'].to_pandas()
data_df_se= data['test_detection_swedish'].to_pandas()


Generating test_detection_swedish split:   0%|          | 0/119 [00:00<?, ? examples/s]

Generating test_detection_english split:   0%|          | 0/119 [00:00<?, ? examples/s]

dict_keys(['test_detection_swedish', 'test_detection_english'])


In [ ]:
data_df_en

,id,source,hyp1,hyp2
0,0,It has enabled us to support and encourage an ...,You can support and encourage an exchange of e...,We can support and encourage an exchange of ex...
1,1,"Therefore, I am calling for an increase in the...",I want to see a decrease in the premiums for a...,I want to see an increase in the premiums for ...
2,2,"In other words, a person may be prevented from...",When there is reason to fear that a violent ac...,When there is reason to fear that a violent ac...
3,3,None of my 34 amendments were adopted and my a...,Our arguments against the ridiculous administr...,My arguments against the ridiculous administra...
4,4,The final key feature of the report is its ori...,The final main feature of the report is its or...,The final features of the report are its orien...
...,...,...,...,...
114,114,I am referring to the current vote but in rela...,The vote on Amendment No 98 will take place be...,The vote on Amendment No 98 will take place af...
115,115,When I became a Member of the European Parliam...,I was struck by the abundance of knowledge whe...,I was struck by the lack of knowledge when I j...
116,116,"Mr President, let me express my thanks for the...","Mr President, I would like to extend a warm we...","Madam President, I would like to extend a warm..."
117,117,We can replace 16 out of 626 Members of the Eu...,"As voters we cannot decide on a new law, but w...","As voters decide on a new law, we can replace ..."


In [ ]:
data_df_se

,id,source,hyp1,hyp2
0,0,Kvinnor kommer att möta högre bilförsäkringspr...,Det betyder att kvinnor kan förvänta sig att b...,Kvinnor kommer att få högre premier för bilför...
1,1,"Rörelseintäkterna var 1,45 miljarder dollar, e...","Intäkterna från rörelsen var 1,45 miljarder do...","Intäkterna från rörelsen var 1,45 miljarder do..."
2,2,"Mandela tillbaka på sjukhus i ""allvarligt men ...",Mandela återvänder inte till sjukhuset med all...,Mandela återvänder till sjukhuset med allvarli...
3,3,Egypten fryser Muslimska brödraskapets tillgån...,Den egyptiska regeringen har fryst tillgångar ...,Egypten fryser tillgångarna för Muslimska bröd...
4,4,"P-pillret, som de kallar ""polypiller"", skulle ...","Medicinen som vanligtvis kallas ""poly pill"" sä...","Medicinen som vanligtvis kallas ""poly pill"" sä..."
...,...,...,...,...
114,114,Texaner som dignar under skyhöga hemförsäkring...,Invånare i Texas som har belastats med orimlig...,Invånare i Texas som har belastats med orimlig...
115,115,Uppdaterad - Två explosioner nära mållinjen i ...,Uppdatering: Nära målet i Boston Marathon intr...,Två explosioner inträffade under loppet av Bos...
116,116,Anthony Weiner halkar ned till fjärde plats i ...,Anthony Weiner halkar till fjärde plats i ny o...,Enligt en ny opinionsundersökning från Quinnip...
117,117,Östtimor förbjuder kampsportsklubbar på grund ...,I Östtimor ska man förbjuda klubbar som utövar...,I Östtimor har man förbjudit klubbar som utöva...


In [ ]:
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')

In [ ]:
os.environ['OPENAI_API_KEY'] = api_key

In [ ]:
from guidance import models, instruction, system, user, assistant
gpt = models.OpenAI("gpt-4-turbo")

# Prompt Version 1

In [ ]:
def detect_hallu(source,hyp1,hyp2):
    # Generate the prompt

    answer_format = {"label": ""}

    user_prompt = f'''
    You are a researcher investigating a new phenomenon. You have gathered data (source) and formulated two competing hypotheses (hyp1 and hyp2) to explain it.
    Identify the hypothesis that contradicts the information provided in the given source.
    Provide the result in the following format: {answer_format}.

    source: {source}
    hyp1 : {hyp1}
    hyp2 : {hyp2}

    Result:
    '''

    with system():
        lm = gpt + "You are a classifier. You specialise in comparing sentences"

    with user():
        lm += user_prompt

    with assistant():
        lm += gen("answer")

    result = lm["answer"]

    print(result)

    return result

In [ ]:
def detect_hallu_se(source,hyp1,hyp2):
    # Generate the prompt

    answer_format = {"label": ""}

    user_prompt = f'''
      Givet en ”src” och två hypoteser ”hyp1” och ”hyp2” är din uppgift att upptäcka vilken av de två hypoteserna (”label”) som inte stöds av källan.
      Ge resultatet i följande format: {answer_format}.

      Src: {source}
      hyp1 : {hyp1}
      hyp2 : {hyp2}

      Resultat:
        '''

    with system():
        lm = gpt + "Du är en klassificerare. Du är specialiserad på att jämföra meningar"

    with user():
        lm += user_prompt

    with assistant():
        lm += gen("answer")

    result = lm["answer"]

    print(result)

    return result

In [ ]:
import json
import re

def extract_json_data(result_text):

    print(result_text)
    # Find the start and end indices of the JSON data
    start_index = result_text.find('{')
    end_index = result_text.find('}') + 1  # Add 1 to include the closing brace

    # Extract the JSON data from the string
    result_text = result_text[start_index:end_index]
    #print("result text: ", result_text)

    # Replace single quotes used as delimiters with double quotes
    result_text = json.loads(result_text.replace("'", "\""))
    print("result_text:",result_text)
    return result_text

In [ ]:
# Set display options to show full content of columns and rows
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)


In [ ]:
# Print the 107th row with columns 'source', 'hyp1', and 'hyp2'
print(data_df_en.loc[108, ['source', 'hyp1', 'hyp2']])

source    I had proposed to you that the Euro-11 be recognised in the Treaty, but my understanding was that Mr Spiers did not agree.
hyp1                                        I wanted the Euro-11 to be recognised in the Treaty but Mr. Spiers didn't agree with me.
hyp2                                              I wanted the Euro-11 to be recognised in the Treaty and Mr. Spiers agreed with me.
Name: 108, dtype: object


In [ ]:
source = "I had proposed to you that the Euro-11 be recognised in the Treaty, but my understanding was that Mr Spiers did not agree."
hyp1= "I wanted the Euro-11 to be recognised in the Treaty but Mr. Spiers didn't agree with me."
hyp2="I wanted the Euro-11 to be recognised in the Treaty and Mr. Spiers agreed with me."
result = detect_hallu(source,hyp1,hyp2)
print()
print(result)


{'label': 'hyp2'}

{'label': 'hyp2'}


## Generate dataset

In [ ]:

import os
import numpy as np
import pandas as pd

def save_results(generate_paraphrase_func, df, csv_filename):

    # Check if the CSV file exists
    if os.path.exists(csv_filename):
        # Read the CSV file using pandas
        existing_csv = pd.read_csv(csv_filename)

        # Check if the DataFrame is not empty
        if not existing_csv.empty:
            # Get the highest ID from the DataFrame
            next_id = existing_csv['id'].max() + 1

            remainder_df = df[df['id'] >= next_id]

            chunks = np.array_split(remainder_df, len(remainder_df) // 10 if len(remainder_df) % 10 == 0 else len(remainder_df) // 10 + 1)
    else:
        # If the CSV file doesn't exist, set highest_id to 1
        next_id = 0

        chunks = np.array_split(df, len(df) // 10 if len(df) % 10 == 0 else len(df) // 10 + 1)

    print("Highest ID: ", next_id)
    # Split DataFrame into chunks of 10 rows

    for chunk in chunks:
        # Iterate through each row in the chunk
        for index, row in chunk.iterrows():
            source = row['source']
            hyp1 = row["hyp1"]
            hyp2 = row["hyp2"]

            try:
              # Generate paraphrase

              result = detect_hallu(source, hyp1, hyp2)

              print(result)
              extracted_hallu = extract_json_data(result)

              hallu_label = extracted_hallu["label"]


              # Update DataFrame with new values
              chunk.at[index, "prediction"] = hallu_label
            except:
                print(f"Skipping row {index} due to error in data extraction.")
                continue

        # Append chunk to CSV file
        if not os.path.isfile(csv_filename):
            chunk.to_csv(csv_filename, mode='w', header=True, index=False)
        else:
            chunk.to_csv(csv_filename, mode='a', header=False, index=False)

        # Load CSV file
        df = pd.read_csv(csv_filename)
        display(df)

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/detection_task_gpt/trial/results_gpt4_en_v1_trial.csv"

save_results(detect_hallu, trial_df_en, csv_filename)

{'label': 'hyp1'}
{'label': 'hyp1'}
{'label': 'hyp1'}
result_text: {'label': 'hyp1'}


,id,source,type,hyp1,hyp2,label,prediction
0,0,"The population has declined in some 210 of the 280 municipalities in Sweden, mainly in inland central and northern Sweden.",antonym,"In the majority of Sweden's 280 municipalities, the population has gone up.","In the majority of Sweden's 280 municipalities, the population has gone down.",hyp1,hyp1
1,1,The draft agenda as drawn up by the Conference of Presidents pursuant to Rule 95 of the Rules of Procedure has been distributed.,negation,The Conference of Presidents hasn't distributed the draft agenda.,The Conference of Presidents has distributed the draft agenda.,hyp1,hyp1
2,2,"I am always grateful for comments and suggestions from the floor, because I believe I need to take everyone's views into account to be an effective President.",antonym,I think I need to listen to everyone's views in order to be an ineffective President.,I think I need to listen to everyone's views in order to be an effective President.,hyp1,hyp1
3,3,The fact is that a key omission from the proposals on agricultural policy in Agenda 2000 is a chapter on renewable energy.,named entity,Agenda 2030 does not include a chapter on renewable energy.,Agenda 2000 does not include a chapter on renewable energy.,hyp1,hyp1
4,4,"We need quite specific legislative proposals, including proposals based on Articles 13 and 137 of the Treaty of Amsterdam.",natural,Legislative proposals based on the Treaty of Amsterdam are needed.,Legislative proposals based solely on the Treaty of Amsterdam are needed.,hyp2,hyp2
5,5,"This state of affairs has not changed in more than 100 years, but hopefully at some stage - and perhaps soon - change will come.",addition,"There has been no change in the status quo in over 100 years, but there is hope that change will soon come.",The state of affairs is1-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-6556,hyp2,hyp2
6,6,"Madam President, I am speaking on behalf of our colleague, Mr Francis Decourrière, who drafted one of the motions for a resolution.",gender,One of the motions for a resolution was drafted by Mr Francis Decourrière.,One of the motions for a resolution was drafted by Mrs Francis Decourrière.,hyp2,hyp2
7,7,Amendment No 1 in the French version deletes illegal immigration and Amendment No 4 omits the expression 'police authorities'.,natural,The French version excludes the expression'police authorities'.,The French version excludes the expression 'police authorities' from Amendment No 4.,hyp1,hyp1
8,8,The European Commission proposes that this information should enter into force within a period of three years from 1 July 1998.,number,The EU wants this information to enter into force in three years.,The EU wants this information to enter into force in thirty years.,hyp2,hyp2
9,9,"Mr President, I did prepare a speech but I have left it aside because many of my points have already been excellently made by previous speakers.",pronoun,"Many of the points they were going to make in their speech were already made by previous speakers, so they left it aside.","Many of the points I was going to make in my speech were already made by previous speakers, so I left it aside.",hyp1,hyp1


In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/detection_task_gpt/trial/results_gpt4_se_v1_trial.csv"

save_results(detect_hallu, trial_df_se, csv_filename)

{'label': 'hyp1'}
{'label': 'hyp1'}
{'label': 'hyp1'}
result_text: {'label': 'hyp1'}


,id,source,type,hyp1,hyp2,label,prediction
0,0,"Men intäkterna från mjukvarulicenser, ett mått som finansanalytiker följer noga, minskade med 21 procent till 107,6 miljoner dollar.",number,"Intäkter från programvarulicenser, en metrik som noggrant övervakas av finansiella analytiker, minskade med 21 procent till ett belopp av 107,6 miljoner dollar.","Intäkter från programvarulicenser, en metrik som noggrant övervakas av finansiella analytiker, minskade med 42 procent till ett belopp av 107,6 miljoner dollar.",hyp2,hyp2
1,1,"Hongkong-universitet samarbetar med universitet, företag och statliga sektorer i Kina för att samordna utbildningsprogram och forskningscentra för att främja högteknologisk forskning, kommersialisering och tekniköverföring.",natural,"University of Hong Kong samarbetar med olika kinesiska universitet, företag och regeringsdepartement för att samordna utbildningsprogram och forskningscentra med syftet att främja avancerad teknologisk forskning, kommersialisering och tekniköverföring.","University of Hong Kong samarbetar med olika kinesiska universitet, företag och statliga sektorer för att samordna utbildningsprogram och forskningscentra med syftet att främja avancerad teknologisk forskning, kommersialisering och tekniköverföring.",hyp1,hyp1
2,2,"Län med befolkningsminskning kommer att vara Vermillion, Posey och Madison.",named entity,"Vermillion, Posey och Madison är län som kommer att uppleva minskande befolkning.","Vermillion, Posey och Marion är län som kommer att uppleva minskande befolkning.",hyp2,hyp2
3,3,Google presenterar en prototyp för en självkörande bil.,addition,En prototyp för en självkörande bil presentera av Google.,"Enligt ett blogginlägg från Google har företaget utvecklat en prototyp av en självkörande bil som kan köra helt på egen hand. Bilen, som kallas ""Waymo One"", har testats i Nevada-öknen och har visat att den kan navigera i komplexa trafikmiljöer utan att behöva någon mänsklig övervakning.",hyp2,hyp1
4,4,Lagförslaget säger att en kvinna som genomgår en sådan abort inte kunde åtalas.,negation,Förslaget stadgar att en kvinna som genomgår en sådan abort kan åtalas.,Förslaget stadgar att en kvinna som genomgår en sådan abort inte kan åtalas.,hyp1,hyp1
5,5,Spaniens prinsessa vittnar i historisk bedrägeriundersökning.,gender,Spanska prinsessan blir en del av en utredning om historisk bedrägeri.,Spanska prinsen blir en del av en utredning om historisk bedrägeri.,hyp2,hyp2
6,6,Mannen använder en slägga för att bryta betongblocket som finns på den andre mannen.,antonym,Mannen använder en slägga för att krossa betongblocket som ligger ovanpå den andra mannen.,Mannen använder en slägga för att krossa betongblocket som ligger under den andra mannen.,hyp2,hyp2
7,7,Nordkorea carnar utlänningar i söder och ger råd om evakuering.,negation,Nordkorea varnar inte utländska medborgare i södra delarna av landet och ger råd om evakuering.,Nordkorea varnar utländska medborgare i södra delarna av landet och ger råd om evakuering.,hyp1,hyp1
8,8,Beväpnad man bland 7 döda efter lägenhetsskjutning i Florida.,addition,En beväpnad man var bland de sju döda efter en lägenhetsskjutning i Florida.,"En man med ett skjutvapen har dödat minst sju personer och skadat flera andra efter att ha öppnat eld i en lägenhet i Fort Lauderdale, Florida, tidigt på morgonen lokal tid. Enligt lokala myndigheter var offren för skjutningen hemmahörande i olika delar av landet. Den misstänkta skytten är gripen. Det är för närvarande oklart vad som motiverade dådet.",hyp2,hyp1
9,9,Irans kärnvapenförhandlingar går in på tredje dagen.,number,Diskussioner om irans kärnvapenprogram har nått sin tredje dag.,Diskussioner om irans kärnvapenprogram har nått sin fjärde dag.,hyp2,hyp2


In [ ]:
import pandas as pd
from sklearn.metrics import classification_report

df = pd.read_csv("/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/detection_task_gpt/trial/results_gpt4_en_v1_trial.csv")

display(df)

# Erstelle den Klassifikationsbericht
report = classification_report(df['label'], df['prediction'])

print(report)

# Speichere den Klassifikationsbericht in eine Textdatei
with open("/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/detection_task_gpt/trial/classification_report_gpt4_en_v1_trial.txt", "w") as text_file:
    text_file.write(report)

,id,source,type,hyp1,hyp2,label,prediction
0,0,"The population has declined in some 210 of the 280 municipalities in Sweden, mainly in inland central and northern Sweden.",antonym,"In the majority of Sweden's 280 municipalities, the population has gone up.","In the majority of Sweden's 280 municipalities, the population has gone down.",hyp1,hyp1
1,1,The draft agenda as drawn up by the Conference of Presidents pursuant to Rule 95 of the Rules of Procedure has been distributed.,negation,The Conference of Presidents hasn't distributed the draft agenda.,The Conference of Presidents has distributed the draft agenda.,hyp1,hyp1
2,2,"I am always grateful for comments and suggestions from the floor, because I believe I need to take everyone's views into account to be an effective President.",antonym,I think I need to listen to everyone's views in order to be an ineffective President.,I think I need to listen to everyone's views in order to be an effective President.,hyp1,hyp1
3,3,The fact is that a key omission from the proposals on agricultural policy in Agenda 2000 is a chapter on renewable energy.,named entity,Agenda 2030 does not include a chapter on renewable energy.,Agenda 2000 does not include a chapter on renewable energy.,hyp1,hyp1
4,4,"We need quite specific legislative proposals, including proposals based on Articles 13 and 137 of the Treaty of Amsterdam.",natural,Legislative proposals based on the Treaty of Amsterdam are needed.,Legislative proposals based solely on the Treaty of Amsterdam are needed.,hyp2,hyp2
5,5,"This state of affairs has not changed in more than 100 years, but hopefully at some stage - and perhaps soon - change will come.",addition,"There has been no change in the status quo in over 100 years, but there is hope that change will soon come.",The state of affairs is1-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-6556,hyp2,hyp2
6,6,"Madam President, I am speaking on behalf of our colleague, Mr Francis Decourrière, who drafted one of the motions for a resolution.",gender,One of the motions for a resolution was drafted by Mr Francis Decourrière.,One of the motions for a resolution was drafted by Mrs Francis Decourrière.,hyp2,hyp2
7,7,Amendment No 1 in the French version deletes illegal immigration and Amendment No 4 omits the expression 'police authorities'.,natural,The French version excludes the expression'police authorities'.,The French version excludes the expression 'police authorities' from Amendment No 4.,hyp1,hyp1
8,8,The European Commission proposes that this information should enter into force within a period of three years from 1 July 1998.,number,The EU wants this information to enter into force in three years.,The EU wants this information to enter into force in thirty years.,hyp2,hyp2
9,9,"Mr President, I did prepare a speech but I have left it aside because many of my points have already been excellently made by previous speakers.",pronoun,"Many of the points they were going to make in their speech were already made by previous speakers, so they left it aside.","Many of the points I was going to make in my speech were already made by previous speakers, so I left it aside.",hyp1,hyp1


              precision    recall  f1-score   support

        hyp1       1.00      1.00      1.00         9
        hyp2       1.00      1.00      1.00         7

    accuracy                           1.00        16
   macro avg       1.00      1.00      1.00        16
weighted avg       1.00      1.00      1.00        16



In [ ]:
import pandas as pd
from sklearn.metrics import classification_report

df = pd.read_csv("/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/detection_task_gpt/trial/results_gpt4_se_v1_trial.csv")

display(df)

# Erstelle den Klassifikationsbericht
report = classification_report(df['label'], df['prediction'])

print(report)

# Speichere den Klassifikationsbericht in eine Textdatei
with open("/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/detection_task_gpt/trial/classification_report_gpt4_se_v1_trial.txt", "w") as text_file:
    text_file.write(report)

,id,source,type,hyp1,hyp2,label,prediction
0,0,"Men intäkterna från mjukvarulicenser, ett mått som finansanalytiker följer noga, minskade med 21 procent till 107,6 miljoner dollar.",number,"Intäkter från programvarulicenser, en metrik som noggrant övervakas av finansiella analytiker, minskade med 21 procent till ett belopp av 107,6 miljoner dollar.","Intäkter från programvarulicenser, en metrik som noggrant övervakas av finansiella analytiker, minskade med 42 procent till ett belopp av 107,6 miljoner dollar.",hyp2,hyp2
1,1,"Hongkong-universitet samarbetar med universitet, företag och statliga sektorer i Kina för att samordna utbildningsprogram och forskningscentra för att främja högteknologisk forskning, kommersialisering och tekniköverföring.",natural,"University of Hong Kong samarbetar med olika kinesiska universitet, företag och regeringsdepartement för att samordna utbildningsprogram och forskningscentra med syftet att främja avancerad teknologisk forskning, kommersialisering och tekniköverföring.","University of Hong Kong samarbetar med olika kinesiska universitet, företag och statliga sektorer för att samordna utbildningsprogram och forskningscentra med syftet att främja avancerad teknologisk forskning, kommersialisering och tekniköverföring.",hyp1,hyp1
2,2,"Län med befolkningsminskning kommer att vara Vermillion, Posey och Madison.",named entity,"Vermillion, Posey och Madison är län som kommer att uppleva minskande befolkning.","Vermillion, Posey och Marion är län som kommer att uppleva minskande befolkning.",hyp2,hyp2
3,3,Google presenterar en prototyp för en självkörande bil.,addition,En prototyp för en självkörande bil presentera av Google.,"Enligt ett blogginlägg från Google har företaget utvecklat en prototyp av en självkörande bil som kan köra helt på egen hand. Bilen, som kallas ""Waymo One"", har testats i Nevada-öknen och har visat att den kan navigera i komplexa trafikmiljöer utan att behöva någon mänsklig övervakning.",hyp2,hyp1
4,4,Lagförslaget säger att en kvinna som genomgår en sådan abort inte kunde åtalas.,negation,Förslaget stadgar att en kvinna som genomgår en sådan abort kan åtalas.,Förslaget stadgar att en kvinna som genomgår en sådan abort inte kan åtalas.,hyp1,hyp1
5,5,Spaniens prinsessa vittnar i historisk bedrägeriundersökning.,gender,Spanska prinsessan blir en del av en utredning om historisk bedrägeri.,Spanska prinsen blir en del av en utredning om historisk bedrägeri.,hyp2,hyp2
6,6,Mannen använder en slägga för att bryta betongblocket som finns på den andre mannen.,antonym,Mannen använder en slägga för att krossa betongblocket som ligger ovanpå den andra mannen.,Mannen använder en slägga för att krossa betongblocket som ligger under den andra mannen.,hyp2,hyp2
7,7,Nordkorea carnar utlänningar i söder och ger råd om evakuering.,negation,Nordkorea varnar inte utländska medborgare i södra delarna av landet och ger råd om evakuering.,Nordkorea varnar utländska medborgare i södra delarna av landet och ger råd om evakuering.,hyp1,hyp1
8,8,Beväpnad man bland 7 döda efter lägenhetsskjutning i Florida.,addition,En beväpnad man var bland de sju döda efter en lägenhetsskjutning i Florida.,"En man med ett skjutvapen har dödat minst sju personer och skadat flera andra efter att ha öppnat eld i en lägenhet i Fort Lauderdale, Florida, tidigt på morgonen lokal tid. Enligt lokala myndigheter var offren för skjutningen hemmahörande i olika delar av landet. Den misstänkta skytten är gripen. Det är för närvarande oklart vad som motiverade dådet.",hyp2,hyp1
9,9,Irans kärnvapenförhandlingar går in på tredje dagen.,number,Diskussioner om irans kärnvapenprogram har nått sin tredje dag.,Diskussioner om irans kärnvapenprogram har nått sin fjärde dag.,hyp2,hyp2


              precision    recall  f1-score   support

        hyp1       0.75      1.00      0.86         9
        hyp2       1.00      0.73      0.84        11

    accuracy                           0.85        20
   macro avg       0.88      0.86      0.85        20
weighted avg       0.89      0.85      0.85        20



In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/detection_task_gpt/results_gpt4_en_v2.csv"

save_results(detect_hallu, data_df_en, csv_filename)

{'label': 'hyp2'}
{'label': 'hyp2'}
{'label': 'hyp2'}
result_text: {'label': 'hyp2'}


,id,source,hyp1,hyp2,prediction
0,0,It has enabled us to support and encourage an exchange of experiences and to pursue activities to raise the level of competence throughout Europe.,You can support and encourage an exchange of experiences to raise the level of competence in Europe.,We can support and encourage an exchange of experiences to raise the level of competence in Europe.,NaN
1,1,"Therefore, I am calling for an increase in the premiums for all varieties of leaf tobacco for the 1999, 2000 and 2001 harvests.",I want to see a decrease in the premiums for all varieties of leaf tobacco.,I want to see an increase in the premiums for all varieties of leaf tobacco.,hyp1
2,2,"In other words, a person may be prevented from coming near their victim when there is reason to fear that violent acts will be carried out again.","When there is reason to fear that a violent act will be carried out again, a person may not be allowed to come near their victim.","When there is reason to fear that a violent act will be carried out again, a person may not be allowed to come near her victim.",NaN
3,3,None of my 34 amendments were adopted and my arguments against the ridiculous administrative burden proposed were not heeded.,Our arguments against the ridiculous administrative burden were not heard.,My arguments against the ridiculous administrative burden were not heard.,hyp1
4,4,"The final key feature of the report is its orientation. It aims to protest against certain drifts that are becoming evident, and I will mention two instances.","The final main feature of the report is its orientation protesting against drifts, and I will provide two instances of that.",The final features of the report are its orientation and two instances.,hyp2
5,5,It is high time an appropriate level of social protection was created at European level for those whose work does not fit into the usual pattern.,It's time for an appropriate level of social protection for those whose work doesn't fit into the usual pattern.,"It's time for an appropriate level of social protection for those whose work doesn't fit into the usual pattern, such as firefighters or security officers.",hyp2
6,6,"These sums, however, have still not produced the desired effect: the substantial fall in the average earnings of the Palestinians is proof of this.",The fall in average earnings of Egyptians is proof that these sums have not produced the desired effect.,The fall in average earnings of Palestinians is proof that these sums have not produced the desired effect.,hyp1
7,7,"Mr President, the approach adopted by the rapporteur to the Commission's 1999 annual economic report is comprehensive and also sensible.",The approach taken by the rapporteur to the 1997 annual economic report is comprehensive and sensible.,The approach taken by the rapporteur to the 1999 annual economic report is comprehensive and sensible.,hyp1
8,8,"The Berlin summit, which will focus solely on Agenda 2000, is of particular importance for the future of the European Union.",The future of the European Union is important to the Berlin summit.,The Berlin summit is important to the future of the European Union.,hyp1
9,9,"As a result, I cannot promise the chairman of the Committee on Budgets that there will be 100 % funding for the financial programme.",I can't promise that there will be 100 percent funding for the financial program.,"Because of this, it's not possible for me to promise the chairman of the Committee on Budgets that there would be 210 % funding for the financial programme.",hyp2


In [ ]:
# Load the CSV file
df = pd.read_csv("/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/detection_task_gpt/results_gpt4_en_v2.csv")

# Select only the columns 'id', 'hyp+', and 'hyp-'
df = df[['id', 'prediction']]
df = df.rename(columns={'prediction': 'label'})

# Save the new DataFrame to a CSV file
df.to_csv('/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/final_gpt4_en_v2_detection.csv', index=False)

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/detection_task_gpt/results_gpt4_se_v1.csv"

save_results(detect_hallu, data_df_se, csv_filename)

{'label': 'hyp1'}
{'label': 'hyp1'}
{'label': 'hyp1'}
result_text: {'label': 'hyp1'}


,id,source,hyp1,hyp2,prediction
0,0,Kvinnor kommer att möta högre bilförsäkringspremier.,Det betyder att kvinnor kan förvänta sig att betala högre priser för sina fordonstilläggsförsäkringar.,Kvinnor kommer att få högre premier för bilförsäkring.,hyp1
1,1,"Rörelseintäkterna var 1,45 miljarder dollar, en ökning jämfört med förra årets resultat på 1,38 miljarder dollar.","Intäkterna från rörelsen var 1,45 miljarder dollar, en minskning jämfört med föregående års resultat på 1,38 miljarder dollar.","Intäkterna från rörelsen var 1,45 miljarder dollar, en ökning jämfört med föregående års resultat på 1,38 miljarder dollar.",hyp1
2,2,"Mandela tillbaka på sjukhus i ""allvarligt men stabilt"" tillstånd.","Mandela återvänder inte till sjukhuset med allvarligt, men stabilt, hälsotillstånd.","Mandela återvänder till sjukhuset med allvarligt, men stabilt, hälsotillstånd.",hyp1
3,3,Egypten fryser Muslimska brödraskapets tillgångar.,"Den egyptiska regeringen har fryst tillgångar för det muslimska brödraskapet, en islamistisk grupp som är förbjuden i Egypten. Detta drag kommer efter att militären störtade president Muhammad Mursi förra året och anklagade gruppen för att planera en kupp. Regeringen hävdar att frysningen av tillgångarna är ett sätt att förhindra att brödraskapet flyr utomlands och försöker störta regeringen igen.",Egypten fryser tillgångarna för Muslimska brödraskapet.,hyp2
4,4,"P-pillret, som de kallar ""polypiller"", skulle innehålla acetylsalicylsyra, ett kolesterolsänkande läkemedel, tre blodtryckssänkande läkemedel vid halva standarddosen och folsyra.","Medicinen som vanligtvis kallas ""poly pill"" sägs innehålla acetylsalicylsyra, en kolesterolsänkande medicin, tre blodtryckssänkande mediciner vid hälften av standarddosen, samt folsyra.","Medicinen som vanligtvis kallas ""poly pill"" sägs innehålla acetylsalicylsyra, en kolesterolsänkande medicin, 2 blodtryckssänkande mediciner vid hälften av standarddosen, samt folsyra.",hyp2
5,5,Beväpnad man dödar 6 i skottlossning mot Wisconsins sikh-tempel.,En beväpnad individ dödar sex personer under en skottlossning vid ett sikh-tempel i Wisconsin.,Två beväpnade individer dödar fyra personer under en skottlossning vid ett sikh-tempel i Wisconsin.,hyp2
6,6,Man som satte eld på sig själv på National Mall dör av skador.,Den person som antände sig själv på National Mall dog till följd av de skador hon åsamkade sig.,Den person som antände sig själv på National Mall dog till följd av de skador han åsamkade sig.,hyp1
7,7,Romney tillkännager Paul Ryan som vicepresidentkandidat.,Romney offentliggör att Paul Ryan är hans utvalde kandidat till posten som vicepresident.,Romney offentliggör att Paul Ryan är hennes utvalde kandidat till posten som vicepresident.,hyp2
8,8,"Läkare säger att en eller båda pojkarna kan dö, och att vissa hjärnskador är möjliga om de överlever.","Läkaren konstaterar att en eller båda pojkarna kan dö, och det finns risk för hjärnskador om de överlever.","Läkaren konstaterar att en eller båda pojkarna är i livsfara, och det finns risk för hjärnskador om de överlever.",hyp2
9,9,Nordkorea säger att en amerikansk medborgare ska ställas inför rätta.,Enligt Nordkorea kommer en medborgare från USA att bli ställd inför rätta.,Enligt Sydkorea kommer en medborgare från USA att bli ställd inför rätta.,hyp2


In [ ]:
# Load the CSV file
df = pd.read_csv("/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/detection_task_gpt/results_gpt4_se_v1.csv")

# Select only the columns 'id', 'hyp+', and 'hyp-'
df = df[['id', 'prediction']]
df = df.rename(columns={'prediction': 'label'})

# Save the new DataFrame to a CSV file
df.to_csv('/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/final_gpt4_se_v1_detection.csv', index=False)

In [ ]:
# update failed rows
def process_csv(file_path, generate_paraphrase_func, prompt):
    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(file_path)

    # Initialize an empty list to store IDs
    missing_ids = []

    # Iterate over each row
    for index, row in df.iterrows():
        # Check if any of the specified columns are empty
        if pd.isna(row['prompt']) and pd.isna(row['hyp+']) and pd.isna(row['hyp-']):
            # Update the "source" and "hyp+" columns with the specified text
            source = row['source']

            missing_ids.append(row['id'])
            try:
              # Generate paraphrase
              result = generate_paraphrase_func(source)
              extracted_hyp = extract_json_data(result)

              hyp_support = extracted_hyp['hyp+']
              print("Hyp+ : ", hyp_support)
              hyp_hallu = extracted_hyp['hyp-']
              print("Hyp- : ", hyp_hallu)

              print('source: ',source)

              # Update DataFrame with new values
              df.at[index, 'hyp+'] = hyp_support
              df.at[index, 'hyp-'] = hyp_hallu
              df.at[index, 'prompt'] = prompt

              print("ADDED")
            except:
                print(f"Skipping row {index} due to error in data extraction.")
                continue

    rows_with_missing_ids = df[df['id'].isin(missing_ids)]

    display(rows_with_missing_ids)

    df.to_csv(file_path, header=True, index=False)

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group2_results/generation_task/generation_task_gpt35/results_gpt35_en.csv"

prompt = '''
    Given the src below, generate a paraphrase hypothesis  hyp+ that is supported by src and a second paraphrase hyp- that is not supported by src.

    Provide the result in the following format: {answer_format}

    Src: {source}

    Result:
'''

result = process_csv(csv_filename, generate_paraphrase, prompt)
result

,id,source,hyp+,hyp-,prompt


In [ ]:
# save a csv file in the task format

import pandas as pd

# Load the CSV file
df = pd.read_csv('/content/drive/MyDrive/Master Project-Hallucination/Group2_results/generation_task/generation_task_gpt4/results_gpt4_en.csv')

# Select only the columns 'id', 'hyp+', and 'hyp-'
df = df[['id', 'hyp+', 'hyp-']]

# Save the new DataFrame to a CSV file
df.to_csv('/content/drive/MyDrive/Master Project-Hallucination/Group2_results/final_gpt4_en.csv', index=False)

# Swedish

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group2_results/generation_task/generation_task_gpt35/results_gpt35_se.csv"

prompt = '''
    användare
    Med tanke på källan nedan, generera en parafras-hypotes hyp+ som stöds av källan och en andra parafras hyp- som inte stöds av källan.
    Ge resultatet i följande format: {answer_format}
    Källa: {source}
    Resultat:
      '''

save_results(generate_paraphrase_se, prompt, data_df_se, csv_filename)

NameError: name 'generate_paraphrase_se' is not defined

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group2_results/generation_task/generation_task_gpt35/results_gpt35_se.csv"

prompt = '''
    användare
    Med tanke på källan nedan, generera en parafras-hypotes hyp+ som stöds av källan och en andra parafras hyp- som inte stöds av källan.
    Ge resultatet i följande format: {answer_format}
    Källa: {source}
    Resultat:
      '''

result = process_csv(csv_filename, generate_paraphrase_se, prompt)
result

NameError: name 'process_csv' is not defined

In [ ]:
# Load the CSV file
df = pd.read_csv("/content/drive/MyDrive/Master Project-Hallucination/Group2_results/generation_task/generation_task_gpt35/results_gpt35_se.csv")

# Select only the columns 'id', 'hyp+', and 'hyp-'
df = df[['id', 'hyp+', 'hyp-']]

# Save the new DataFrame to a CSV file
df.to_csv('/content/drive/MyDrive/Master Project-Hallucination/Group2_results/final_gpt35_se.csv', index=False)